# Working with Milvus and LangChain

In [19]:
## Let's go to work with LangChain and Milvus

### Setup LangChain, Milvus, and Embeddings imports

In [21]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Milvus
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader


In [45]:
# Load something simple and split it 

loader = TextLoader('state_of_the_union.txt')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

In [40]:
# print(f'{docs}')

In [46]:
# Generate embeddings with your favorite library (First time this will download model if need be)

embeddings = SentenceTransformerEmbeddings()

In [51]:
# Connect and Embed
vector_db = Milvus.from_documents(
    docs,
    embeddings,
    collection_name="state_of_union_01",
    connection_args={"host": "0.0.0.0", "port": "19530"},
)


In [58]:
## Explore your embeddings

# print(vector_db._collection.count()) #     collection_properties)    #collection_properties()) #   _collection.count())
query = "What is this document about?"

docs = vector_db.similarity_search(query)

# print(f"{print(docs[1].page_content)}")

In [60]:
# Setup a LangChain Retriever and Chain (assumes ollama locallly running mistral)

In [62]:
#### Model 

# OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", cfg.OLLAMA_BASE_URL)
# OLLAMA_MODEL = "mistral"

from langchain_community.chat_models import ChatOllama

model = ChatOllama(
    model='mistral',
    base_url='http://localhost:11434',
)


In [65]:
### Prompt

from langchain.prompts import ChatPromptTemplate

template = """Answer the question giving priority to the following context if it is relevant.
    If what follows does not answer the question silently ignore the following context and answer using your own knowledge:

    {context}

    Question: {question}
    """
prompt = ChatPromptTemplate.from_template(template)

In [69]:
### Optional (Output Parser)

from langchain.schema import StrOutputParser

output_parser = StrOutputParser()

In [71]:
# Now connect LangChain to Milvus

vector_db = Milvus.from_documents(
    docs,
    embeddings,
    collection_name='state_of_union_0',
    connection_args={'host': '0.0.0.0', 'port': '19530'},
)


In [72]:
### Retriever (ie get embeddings from Milvus)

retriever = vector_db.as_retriever() #     docs.as_retriever()


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

NameError: name 'format_docs' is not defined